In [ ]:
 ============================================================================================
                                         DATA INGESTION
 ============================================================================================

In [1]:
# importing the appropraite library
import pandas as pd
import json

In [2]:
# Flattening json file for proper viewing
file_path = "C:/Oluwatosin_Git/Real_Estate_Data_Pipeline/data/raw_data/2025-11-18.json"

with open(file_path) as f:
    data = json.load(f)
df = pd.json_normalize(data["data"])
homes = df

In [ ]:
 ============================================================================================
                                        DATA TRANSFORMATION
 ============================================================================================

In [14]:
# ------------------------ TABLE PREPARATION ---------------------------
rows = []
column_names = ["property_id", "home_address", "state", "city", "zipcode", "latitude",
                "longitude", "bedrooms", "bathrooms", "living_area", "lot_size", 
                "year_built", "property_type", "price", "price_sqft", "status",
                "days_on_zillow"]

In [16]:
# ---------------------------- TABLE FIILING ----------------------------
# Looping through the data'
for index, home in homes.iterrows():
    property_id =       home["zpid"]
    home_address =      home["address.streetAddress"]
    state =             home["address.state"]
    city =              home["address.city"]
    zipcode =           home["address.zipcode"]
    latitude =          home["location.latitude"]
    longitude =         home["location.longitude"]
    bathrooms =         home["bathrooms"]
    bedrooms =          home["bedrooms"]
    living_area =       home["livingArea"]
    lot_size =          home["lotSizeWithUnit.lotSize"]
    year_built =        home["yearBuilt"]
    property_type =     home["propertyType"]
    price =             home["price.value"]
    price_sqft =        home["price.pricePerSquareFoot"]
    status =            home["listing.marketingStatus"]
    days_on_zillow =    home["daysOnZillow"]

    tuple_of_records = (property_id, home_address, state, city, zipcode, latitude,
                longitude, bedrooms, bathrooms, living_area, lot_size, 
                year_built, property_type, price, price_sqft, status,
                days_on_zillow)
    
    rows.append(tuple_of_records)

In [17]:
# Printing into dataframe
print(rows)


[(28208218, '4122 Great Bear Pl', 'TX', 'Houston', '77084', 29.833529, -95.69997, 4, 2.0, 2611, 7065.432, 2019, 'singleFamily', 389010, 149, 'active', 26), (28208218, '15322 Torry Pines Rd', 'TX', 'Houston', '77062', 29.575647, -95.13027, 4, 2.0, 2015, 9840.204, 1970, 'singleFamily', 369000, 183, 'active', 10), (28208218, '2851 Decker Pl', 'TX', 'Houston', '77003', 29.758352, -95.33921, 3, 4.0, 2401, 2134.44, 2018, 'singleFamily', 499999, 208, 'active', 5), (28208218, '717 E 23rd St', 'TX', 'Houston', '77008', 29.807554, -95.39053, 3, 3.0, 1957, 4678.344, 1928, 'singleFamily', 789000, 403, 'active', 13), (28208218, '5711 Ridgeway Dr', 'TX', 'Houston', '77033', 29.66928, -95.33654, 4, 1.0, 1240, 7204.824, 1958, 'singleFamily', 159000, 128, 'active', 3), (28208218, '1815 Wagon Gap Trl', 'TX', 'Houston', '77090', 30.01964, -95.46432, 4, 4.0, 2919, 8969.004, 1975, 'singleFamily', 299900, 103, 'active', 23), (28208218, '12511 Silver Cup', 'TX', 'Houston', '77014', 29.971582, -95.46673, 3, 3

In [39]:
df = pd.DataFrame(rows, columns=column_names)

In [30]:
df.index = df.index + 1

In [37]:
df.head(3)

,property_id,home_address,state,city,zipcode,latitude,longitude,bedrooms,bathrooms,living_area,lot_size,year_built,property_type,price,price_sqft,status,days_on_zillow
1,28208218,4122 Great Bear Pl,TX,Houston,77084,29.833529,-95.69997,4,2.0,2611,7065.432,2019,singleFamily,389010,149,active,26
2,28208218,15322 Torry Pines Rd,TX,Houston,77062,29.575647,-95.13027,4,2.0,2015,9840.204,1970,singleFamily,369000,183,active,10
3,28208218,2851 Decker Pl,TX,Houston,77003,29.758352,-95.33921,3,4.0,2401,2134.440,2018,singleFamily,499999,208,active,5


In [42]:
regularise1 = df["property_type"].unique()
regularise2 = df["status"].unique()
print(regularise1)
print(regularise2)

['singleFamily']
['active']


In [34]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50 entries, 1 to 50
Data columns (total 17 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   property_id     50 non-null     int64  
 1   home_address    50 non-null     object 
 2   state           50 non-null     object 
 3   city            50 non-null     object 
 4   zipcode         50 non-null     object 
 5   latitude        50 non-null     float64
 6   longitude       50 non-null     float64
 7   bedrooms        50 non-null     int64  
 8   bathrooms       50 non-null     float64
 9   living_area     50 non-null     int64  
 10  lot_size        48 non-null     float64
 11  year_built      50 non-null     int64  
 12  property_type   50 non-null     object 
 13  price           50 non-null     int64  
 14  price_sqft      50 non-null     int64  
 15  status          50 non-null     object 
 16  days_on_zillow  50 non-null     int64  
dtypes: float64(4), int64(7), object(6)
me

In [ ]:
---------------------------------------------- DATA CLEANING PROPER ------------------------------------------------

In [54]:
class Zillow_data_cleaner:
    def __init__(self, df):
         self.df = df.copy()
        
    # Function 1: Converting datatypes
    def fix_dtypes(self):
        if 'bathroom' in self.df.columns:
            self.df['bathroom'] = self.df['bathroom'].astype(float)
        if 'bedroom' in self.df.columns:
            self.df['bedroom'] = self.df['bedroom'].astype(int)
        if 'living_area' in self.df.columns:
            self.df['living_area'] = self.df['living_area'].astype(int)
        if 'lot_size' in self.df.columns:
            self.df['lot_size'] = self.df['lot_size'].astype(float)
        if 'price' in self.df.columns:
            self.df['price'] = self.df['price'].astype(int)
        return self
    
    # Function 2: Handling of missing values
    def missing(self):
        # Drop rows where these fields are missing
        self.df = self.df.dropna(subset=['property_id', 'home_address'])
        return self
    
    # Function 3: Remove Duplicates
    def remove_duplicates(self):
        # Drop rows where these fields are missing
        self.df = self.df.drop_duplicates(subset='property_id', keep='first')
        return self
    
    # Function 4: standardize
    def standadize(self):
        self.df["property_type"] = self.df["property_type"].str.replace("singleFamily", "Single Family")
        self.df["status"] = self.df["status"].str.replace("active", "Active")
        self.df["state"] = self.df["state"].str.replace("TX", "Texas")
        return self
    
    # Function 5: Roundingoff
    def rounding_off(self):
        self.df["lot_size"] = self.df["lot_size"].round(1)
        return self
    
    def clean_all(self):
        # Run all the cleaning at a go
        return self.fix_dtypes().missing().remove_duplicates().standadize().rounding_off().df

In [55]:
cleaner = Zillow_data_cleaner(df)
clean_df = cleaner.clean_all()

In [56]:
clean_df.head(3)

,property_id,home_address,state,city,zipcode,latitude,longitude,bedrooms,bathrooms,living_area,lot_size,year_built,property_type,price,price_sqft,status,days_on_zillow
0,28208218,4122 Great Bear Pl,Texas,Houston,77084,29.833529,-95.69997,4,2.0,2611,7065.4,2019,Single Family,389010,149,Active,26
25,457154785,4122 Great Bear Pl,Texas,Houston,77084,29.833529,-95.69997,4,2.0,2611,7065.4,2019,Single Family,389010,149,Active,26
26,28133604,15322 Torry Pines Rd,Texas,Houston,77062,29.575647,-95.13027,4,2.0,2015,9840.2,1970,Single Family,369000,183,Active,10
